# Step 1 – Load Parsed CSV

In [1]:
import pandas as pd

df = pd.read_csv("../data/output_csvs/parsed_resumes.csv")
df.head()


,Name,Education,Experience,Skills,Projects,Certifications,Raw_Text,Filename
0,SR. ESTIMATOR-BAS CONSTRUCTION SALES,educational attainment and professional experi...,experience in energy solution consulting and p...,"Skills\nSales, Proposals, Solutions, Commercia...",projects. Total projects worked for fiscal yea...,Certifications\nEnergy Management Professional...,SR. ESTIMATOR-BAS CONSTRUCTION SALES\nSummary\...,10041713.pdf
1,INFORMATION TECHNOLOGY TECHNICIAN I,education programs to explain upgrades to end ...,"Experienced in server management, systems anal...","skills for networking issues, end user problem...",projects.\nCollaborating on project plans for ...,Certifications\nCompTIA Network+ - 2014\nSkill...,INFORMATION TECHNOLOGY TECHNICIAN I\nSummary\n...,10089434.pdf
2,CARPENTER APPRENTICE,Education\nHigh School Diploma : 04/2010 \nBuf...,experience\nworking with remodeling projects i...,skills. Interested in growing career and foste...,projects in residential and commercial constru...,NaN,CARPENTER APPRENTICE\nProfessional Summary\nFo...,10100240.pdf
3,SALES MANAGER,Education\n2012\nHigh School Diploma : General...,Experience\n01/2010 to Current\nSales Manager ...,Skills\nâ€¢Computers and Electronics\nâ€¢Admin...,NaN,NaN,SALES MANAGER\nProfessional Summary\nAccomplis...,10138632.pdf
4,NEW CONSTRUCTION / AFE MAINTENANCE MANAGER,Education\nHigh School Diploma : Industrial Ma...,"experience in Shutdowns, T/A's, and Project re...","skills="""" and="""" relationships="""" with=""""\ncon...",projects are always at the highest level.Â Pro...,Certifications\nEmergency Response\nActive Fir...,NEW CONSTRUCTION / AFE MAINTENANCE MANAGER\nSu...,10149490.pdf


# Step 2 – Rule-Based Scoring Function

In [2]:
import re

def score_resume(row):
    score = {}
    
    # Skill count
    skills = row.get("Skills", "")
    score["has_enough_skills"] = len(skills.split(',')) >= 5

    # Projects section
    score["has_projects"] = bool(row.get("Projects", "").strip())

    # Experience section: check for action verbs
    experience = row.get("Experience", "")
    verbs = ["led", "developed", "built", "analyzed", "created", "implemented"]
    score["uses_action_verbs"] = any(v in experience.lower() for v in verbs)

    # Impact metrics: %, $, numbers
    score["has_quantified_impact"] = bool(re.search(r"[\$%][0-9]|[0-9]+%", experience))

    # Education / Certs presence
    score["has_education"] = bool(row.get("Education", "").strip())
    score["has_certifications"] = bool(row.get("Certifications", "").strip())

    return score


#  Step 3 – Gemini API Setup

In [4]:
pip install google-generativeai


  Using cached protobuf-6.31.1-cp310-abi3-win_amd64.whl.metadata (593 bytes)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached typing_extensions-4.14.1-py3-none-any.whl.metadata (3.0 kB)
  Using cached requests-2.32.4-py3-none-any.whl.metadata (4.9 kB)
INFO: pip is looking at multiple versions of grpcio-status to determine which version is compatible with other requirements. This could take a while.
  Using cached charset_normalizer-3.4.2-cp313-cp313-win_amd64.whl.metadata (36 kB)
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
  Using cached urllib3-2.5.0-py3-none-any.whl.metadata (6.5 kB)
  Using cached certifi-2025.7.14-py3-none-any.whl.metadata (2.4 kB)
   ---------------------------------------- 0.0/1.3 MB ? eta -:--:--
   ---------------------------------------- 1.3/1.3 MB 8.3 MB/s eta 0:00:00
   ---------------------------------------- 0.0/4.5 MB ? eta -:--:--
   ------------------ --------------------- 2.1/4.5 MB 11.1 MB/s eta 0:00:01
   ---

In [5]:
import google.generativeai as genai

genai.configure(api_key="YOUR_GEMINI_API_KEY")  # Replace with your actual key

model = genai.GenerativeModel("gemini-pro")


c:\Users\rangi\OneDrive\Desktop\resume_analyzer_project\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


#  Step 4 – LLM Feedback Generator

In [7]:
def generate_feedback_gemini(name, experience, projects, skills):
    prompt = f"""
You are an expert resume reviewer. Evaluate this resume content for clarity, completeness, and impact.

Name: {name}

Experience:
{experience}

Projects:
{projects}

Skills:
{skills}

Provide improvement suggestions in bullet points. Highlight anything that is missing or vague.
"""
    try:
        response = model.generate_content(prompt)
        return response.text
    except Exception as e:
        return f"Error: {str(e)}"


#  Step 5 – Apply to All Rows

In [9]:
import json

score_reports = []
feedbacks = []

for idx, row in df.iterrows():
    try:
        # Score using rule-based function
        score_dict = score_resume(row)
        score_reports.append(json.dumps(score_dict))

        # Generate LLM feedback using Gemini
        feedback = generate_feedback_gemini(
            name=str(row.get("Name", "")),
            experience=str(row.get("Experience", "")),
            projects=str(row.get("Projects", "")),
            skills=str(row.get("Skills", ""))
        )
        feedbacks.append(feedback)

        print(f"✅ Processed resume {idx + 1}/{len(df)}")

    except Exception as e:
        print(f"❌ Error in resume {idx + 1}: {e}")
        score_reports.append(json.dumps({"error": "scoring failed"}))
        feedbacks.append(f"LLM error: {str(e)}")


✅ Processed resume 1/186
✅ Processed resume 2/186
❌ Error in resume 3: 'float' object has no attribute 'strip'
❌ Error in resume 4: 'float' object has no attribute 'strip'
✅ Processed resume 5/186
✅ Processed resume 6/186
❌ Error in resume 7: 'float' object has no attribute 'strip'
❌ Error in resume 8: 'float' object has no attribute 'strip'
❌ Error in resume 9: 'float' object has no attribute 'strip'
❌ Error in resume 10: 'float' object has no attribute 'strip'
✅ Processed resume 11/186
❌ Error in resume 12: 'float' object has no attribute 'strip'
❌ Error in resume 13: 'float' object has no attribute 'strip'
❌ Error in resume 14: 'float' object has no attribute 'strip'
❌ Error in resume 15: 'float' object has no attribute 'strip'
❌ Error in resume 16: 'float' object has no attribute 'strip'
❌ Error in resume 17: 'float' object has no attribute 'strip'
❌ Error in resume 18: 'float' object has no attribute 'strip'
✅ Processed resume 19/186
❌ Error in resume 20: 'float' object has no att

# Step 6 – Save Final Results

In [10]:
df["Score_Report"] = score_reports
df["LLM_Feedback"] = feedbacks

df.to_csv("../data/output_csvs/evaluated_resumes.csv", index=False)
print("✅ Evaluation complete. Saved to evaluated_resumes.csv")


✅ Evaluation complete. Saved to evaluated_resumes.csv
